<a href="https://colab.research.google.com/github/poojakabber7/DallE2-ai-generator-scott/blob/main/Non%20DRL%20Recommenders/Dataset1_RetailRocket/SNQNRRitemfeatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!rm -R /content/Deep_Reinforcement_Learning_Recommenders
!git clone https://github.com/kashafali8/Deep_Reinforcement_Learning_Recommenders.git

rm: cannot remove '/content/Deep_Reinforcement_Learning_Recommenders': No such file or directory
Cloning into 'Deep_Reinforcement_Learning_Recommenders'...
remote: Enumerating objects: 580, done.
remote: Counting objects: 100% (74/74), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 580 (delta 44), reused 21 (delta 9), pack-reused 506
Receiving objects: 100% (580/580), 8.65 MiB | 10.92 MiB/s, done.
Resolving deltas: 100% (340/340), done.


In [3]:
!pip install trfl
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 kB 14.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [17]:
!pip install kaggle -q      # At first, I suspect the kaggle API lose effect so it doesn't have .kaggle folder. (not working)
!rm -rf /root/.kaggle.      # when I created the folder, it says the file or dir already exits
!mkdir /root/.kaggle        # successful
!mv /content/kaggle.json /root/.kaggle/kaggle.json    # not sure if I have to use full destination path, I previously only used /root/.kaggle and it failed. Don't have time to validate this thought.
!ls /root/.kaggle/kaggle.json

/root/.kaggle/kaggle.json


In [20]:
!kaggle datasets download -d retailrocket/ecommerce-dataset
!unzip ecommerce-dataset.zip -d "/content/Deep_Reinforcement_Learning_Recommenders/Non DRL Recommenders/Dataset1_RetailRocket/data/"
!rm ecommerce-dataset.zip

100% 290M/291M [00:13<00:00, 24.2MB/s]
100% 291M/291M [00:13<00:00, 22.2MB/s]
Archive:  ecommerce-dataset.zip
  inflating: /content/Deep_Reinforcement_Learning_Recommenders/Non DRL Recommenders/Dataset1_RetailRocket/data/category_tree.csv  
  inflating: /content/Deep_Reinforcement_Learning_Recommenders/Non DRL Recommenders/Dataset1_RetailRocket/data/events.csv  
  inflating: /content/Deep_Reinforcement_Learning_Recommenders/Non DRL Recommenders/Dataset1_RetailRocket/data/item_properties_part1.csv  
  inflating: /content/Deep_Reinforcement_Learning_Recommenders/Non DRL Recommenders/Dataset1_RetailRocket/data/item_properties_part2.csv  


In [21]:
!python "/content/Deep_Reinforcement_Learning_Recommenders/Non DRL Recommenders/Dataset1_RetailRocket/src/replay_buffer.py" --data "/content/Deep_Reinforcement_Learning_Recommenders/Non DRL Recommenders/Dataset1_RetailRocket/data/"


Starting to pre-process data...

Sorting and pickling data...

Splitting data into train, validation, and test sets...

Pickling train, validation, and test sets...

Calculating item popularity and storing as dictionary...

Generating replay buffer from train set...

Pickling replay buffer...

Pickling data statistics...

Script completed successfully!


In [ ]:
import pandas as pd
import numpy as np


def create_feature_matrix(
    sorted_events, n_files=2, path_name="", one_hot_encode=True, top_features=500
):
    for i in range(n_files):
        if i == 0:
            item_features = pd.read_csv(path_name + str(i + 1) + ".csv")
        else:
            item_features = pd.concat(
                [item_features, pd.read_csv(path_name + str(i + 1) + ".csv")],
                ignore_index=True,
            )

    item_features = item_features[
        item_features["itemid"].isin(sorted_events["item_id"].unique().tolist())
    ].drop_duplicates()
    item_features["property_value"] = (
        item_features["property"].str.strip() + item_features["value"].str.strip()
    )
    item_features = item_features.drop(["timestamp"], axis=1).drop_duplicates()

    if one_hot_encode:
        one_hot_encoded = pd.DataFrame()
        itemids = []

        event_item_list = sorted_events.item_id.unique()
        event_item_list.sort()
        item_list = item_features["itemid"].unique()
        properties = (
            item_features["property_value"]
            .value_counts()
            .head(top_features)
            .index.tolist()
        )

        for item in event_item_list:
            if len(itemids) % 1000 == 0:
                print("hi")
            if item not in item_list:
                one_hot_encoded = pd.concat(
                    [one_hot_encoded, pd.DataFrame(np.zeros(len(properties))).T],
                    ignore_index=True,
                )
                itemids.append(item)
                continue

            item_properties = item_features[item_features["itemid"] == item][
                "property_value"
            ].unique()
            one_hot_encoded = pd.concat(
                [
                    one_hot_encoded,
                    pd.DataFrame(
                        [1 if x in item_properties else 0 for x in properties]
                    ).T,
                ],
                ignore_index=True,
            )
            itemids.append(item)

        return one_hot_encoded, itemids

    else:
        return item_features, item_features["itemid"].unique().tolist()

In [ ]:
sorted_events = pd.read_pickle("/content/Deep_Reinforcement_Learning_Recommenders/DRL_Recommenders/Dataset_RR/data/sorted_events.df")
one_hot_encoded, itemids = create_feature_matrix(
    sorted_events, n_files=2, path_name="/content/Deep_Reinforcement_Learning_Recommenders/DRL_Recommenders/Dataset_RR/data/item_properties_part", one_hot_encode=True
)

In [ ]:
one_hot_encoded.to_pickle("/content/Deep_Reinforcement_Learning_Recommenders/DRL_Recommenders/Dataset_RR/data/one_hot_encoded.df")

In [23]:
!python "/content/Deep_Reinforcement_Learning_Recommenders/Non DRL Recommenders/Dataset1_RetailRocket/src/SNQN_IF.py" --model=SASRec --epoch=5 --data="/content/Deep_Reinforcement_Learning_Recommenders/Non DRL Recommenders/Dataset1_RetailRocket/data"

2023-05-05 00:42:44.627274: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-05 00:42:45.591603: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/Deep_Reinforcement_Learning_Recommenders/Non DRL Recommenders/Dataset1_RetailRocket/src/SNQN_IF.py:185: UserWarning: `tf.layers.dropout` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dropout` instead.
  self.seq = tf.compat.v1.layers.dropout(self.seq,
2023-05-05 00:42:49.115270: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Pla